In [1]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
import numpy as np
import os

In [12]:
model = tf.keras.models.load_model(r"C:\xray disease\Human_Eye_Disease_Prediction\Trained_Model.h5")

ValueError: Unknown metric function: 'F1Score'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [9]:
pip install pillow

   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.0 MB ? eta -:--:--
   --------- ------------------------------ 1.6/7.0 MB 7.7 MB/s eta 0:00:01
   ---------------------------- ----------- 5.0/7.0 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 10.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
import numpy as np
import os
import cv2  # Using OpenCV as an alternative to PIL

# Define the F1Score metric class that was used during training
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert probabilities to binary predictions
        y_pred = tf.round(y_pred)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)
        
    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))
        
    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

# Load the model with the custom metric
try:
    model = tf.keras.models.load_model(
        r"C:\xray disease\Human_Eye_Disease_Prediction\Trained_Model.h5",
        custom_objects={'F1Score': F1Score}
    )
    print("Model loaded successfully with F1Score metric")
except Exception as e:
    print(f"Error loading with custom metric: {e}")
    print("Trying to load without compiling...")
    model = tf.keras.models.load_model(
        r"C:\xray disease\Human_Eye_Disease_Prediction\Trained_Model.h5",
        compile=False
    )
    print("Model loaded successfully with compile=False")

# Load and preprocess image using OpenCV (no PIL required)
img_path = r'C:\xray disease\Human_Eye_Disease_Prediction\test\DRUSEN\DRUSEN-1112835-7.jpeg'

# Check if the image file exists
if not os.path.exists(img_path):
    print(f"Error: Image file not found at {img_path}")
else:
    # Load image with OpenCV
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error: Could not load image from {img_path}")
    else:
        # Convert BGR to RGB and resize
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        
        # Convert to array and preprocess
        x = np.array(img, dtype=np.float32)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        
        print(f"Input shape: {x.shape}")
        
        # Make prediction
        preds = model.predict(x)
        
        # Get class names (make sure this path is correct)
        test_dir = "./test"
        if os.path.exists(test_dir):
            class_name = os.listdir(test_dir)
            result_index = np.argmax(preds)
            predicted_class = class_name[result_index]
            
            print(f"Predicted class: {predicted_class}")
            print(f"Confidence scores: {preds}")
            
            # Show confidence for each class
            for i, class_name in enumerate(class_name):
                print(f"{class_name}: {preds[0][i]:.4f}")
        else:
            print(f"Test directory not found at {test_dir}")
            # Use hardcoded class names as fallback
            class_name = ['CNV', 'DME', 'DRUSEN', 'NORMAL']
            result_index = np.argmax(preds)
            predicted_class = class_name[result_index]
            
            print(f"Predicted class: {predicted_class}")
            print(f"Confidence scores: {preds}")
            
            for i, class_name in enumerate(class_name):
                print(f"{class_name}: {preds[0][i]:.4f}")

ModuleNotFoundError: No module named 'cv2'

In [ ]:
img_path = './test/DRUSEN/DRUSEN-1112835-7.jpeg'
img = tf.keras.utils.load_img(img_path, target_size=(224, 224))
x = tf.keras.utils.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
x.shape

(1, 224, 224, 3)

In [ ]:
preds = model.predict(x)

1/1 [==============================] - 0s 31ms/step


In [ ]:
preds

array([[0.07345137, 0.01443424, 0.8707393 , 0.04137512]], dtype=float32)

In [ ]:
class_name = os.listdir("./test")
class_name

['CNV', 'DME', 'DRUSEN', 'NORMAL']

In [ ]:
result_index = np.argmax(preds)
class_name[result_index]

'DRUSEN'